# Regression topics

This section will go into more detail on running regressions in Python. We already saw an example using [factor models](10_factor_models.html#factor_models), like the CAPM and Fama-French 3-factor models. 

We could spend an entire semester going over linear regression, how to put together models, how to interpret models, and all of the adjustments that we can make. In fact, this is basically what a first-semester Econometrics class is!

This chapter is divided into two parts. I start by following code examples from [Coding for Economists](https://aeturrell.github.io/coding-for-economists/econmt-regression.html), which has just about everything you need to know to do basic linear regression (OLS) in Python. This gets us our first look at a linear model and some output.

I then turn to **machine learning** and **Chapters 3 and 10 from our Hull textbook**. These chapters also focus on linear regression and how to interpret these types of model. However, we'll also see some newer regression topics from the machine learning perspective, such as **regularization, Ridge regression, and LASSO regression**.  They also discuss logit models, something that we get into in the next set of notes. 

[The Effect](https://theeffectbook.net) is a great book for getting starting with econometrics, regression, and how to add meaning to the regressions that we're running. [Chapter 13](https://theeffectbook.net/ch-StatisticalAdjustment.html) of that book covers regression (with code in R). Like most econometrics, that book is focused on **causality** - did some intervention or change **cause** a change in behavior or another outcome? These tools are beyond the scope of this course, but we have a new course on Causal Inference if you're interested.

You'll also see categorical variables and logs. Thinking about how to define our variables is an example of **feature engineering** in the machine learning world.

## Supervised Learning and the Machine Learning Process. Hull, Chapter 1.

Chapter 1 of Hull starts us the Hull book starts our discussion of machine learning. Tradtional econometrics is about **explaining**. Machine learning is about **predicting**. Roughly speaking.

We saw **unsupervised learning** when looking at clustering. Regression gets us into **supervised learning**. From Hull, Chapter 1:

> Supervised learning is concerned with using data to make **predictions**. In the next section, we will show how a simple regression model can be used to predict salaries. This is an example of supervised learning. In Chapter 3, we will consider how a similar model can be used to  predict house prices. We can distinguish between supervised learning models used to predict a variable that can take a continuum of values (such as an individual’s salary or the price of a house) and supervised learning models that are used for classification.

> The data for supervised learning contains what are referred to as  features and labels. The labels are the values of the target (e.g., the value  of a house or whether a particular loan was repaid). The features are  the variables from which the predictions about the target are made.

Prediction means that we're going to approach things a bit differently. In particular, we are going to think carefully about **in-sample** vs. **out-of-sample** prediction. From Hull, Chapter 1:

> When a data set is used for forecasting or determining a decision strategy, there is a danger that the machine learning model will work well for the data set but will not generalize well to other data. As statisticians have realized for a long time, it is important to test a model out-of-sample. By this we mean that the model should be tested on data that is different from the data used to determine the parameters of the model. 

Our process is going to have us **fit** a model using **training** data. We might use some **validation** data to **fine-tune** the model. Finally, we **evaluate** the model using our **testing** data.


## statsmodel vs. sklearn

I run linear regressions (OLS) two ways below. I first use `statsmodels`. This method and output looks like "traditional" regression analysis. You specify the y-variable, the x-variables you want to include, and get a nice table with output. Very much like what you'd get using Excel and the Data Analysis Toolkit. 

The Hull textbook uses `sklearn`. This library is focused on **machine learning**. The set-up is going to look a lot different. This library will help you process your data, define features, and split it up into training and test data sets. Crucially, you'll want to end up with **X** and **y** data frames that contain **features** and **target** values that you are trying to predict, respectively.

Two libraries, two ways to do regression. The Hull textbook uses `sklearn` for the **ridge, LASSO, and Elastic Net** models.

You can read more about [the statsmodels library on their help page](https://www.statsmodels.org/dev/regression.html). Here's [linear regression from the sklearn libary](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). 


## Workflow

A lot of the data that we're using in these examples is already clean. There are no missing values. The columns are what we want. The features and target have been standardized. 

In reality, things don't work that way. You need to **carefully look at your data**. The regressions, the models, the machine learning is "easy", in a sense. The tools are built-in. You don't exactly have to do a lot of linear algebra or optimization math by hand. 

The real work is in the details. It is your judgement and domain expertise that tells you what features you should be looking for. You need to know how your data were collected. You need the background knowledge to understand your task.

You'll spend a lot of time cleaning and organizing your data in our labs and exams. In general, your workflow is going to be something like this:

- **Import your data**. Is it a CSV or Excel file? Is it what you want? Do you need to remove any extra columns? Skip rows? 
  
- **Keep the columns that your want.** What's your target, the thing you predicting? What features do you want to include?
  
- **Feature engineering time!** Any missing values? Anything to clean? Check all of you columns. Are some text that you want numeric? Everything we're doing here, for now, is with numeric values. Do you need to create new variables from existing ones? Combine variables into a new feature? Do you want to create dummy variables for categorical features? **You'll spend most of your time doing this.**
  
- **Split your data.** Once your data is clean, you can split it into **training** and **testing** samples. Hull will also sometimes split things into an intermediate **validation** sample. Most of your data will be in the training sample. You'll hold out maybe 20% of your data for testing.
  
- **Standardize your data.** Turn the actual numerical values for your targets and features into z-scores. A general recommendation is to standardize your training data and then use the means and standard deviations from your training data to standardize your testing data. More on this below. 
  
Hull often provides data that is already cleaned, split, and standardized. But, we need to know how to do this. He mentions in the text that he standardizes the testing data using the means and standard deviations from the training data. But, we don't see this when using data that is already cleaned up.
  
- **Train your model.** Use your training data to fit your model. Use a validation data set or something like cross-fold validation to look for the optimal hyperparameter(s), if needed.
  
- **Predict your target.** Take the fitted model that uses the optimal hyperparamter(s) and use it to fit your testing data. Get predicted values and compare to the actual y targets in your testing sample.
  
- **Evaluate your model.** How good a job did it do? See Hull for more on the numbers you might use to evaluate a model. I like to make a scatter plot of actual vs. predicted values. 


In [1]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.linear_model import LinearRegression

# Importing Ridge
from sklearn.linear_model import Ridge

# Import Lasso
from sklearn.linear_model import Lasso

# Import Elastic Net
from sklearn.linear_model import ElasticNet

# Import train_test_split
from sklearn.model_selection import train_test_split

# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

#Need for AIC/BIC cross validation example
from sklearn.linear_model import LassoLarsIC
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold


# Include this to have plots show up in your Jupyter notebook.
%matplotlib inline 

pd.options.display.max_columns = None

## Example using Zillow pricing errors

Let's start with a typical regression and output. So, no prediction. Let's run a regression like we might using the Data Analysis Toolkit in Excel. We'll look at the ouput and interpret the coefficients and statistical significance. 

I'll be using our Zillow pricing error data in this example. The `statsmodel` library will let us run the regressions and give us nicely formatted output.

In [2]:
housing = pd.read_csv('https://raw.githubusercontent.com/aaiken1/fin-data-analysis-python/main/data/properties_2016_sample10_1.csv')
pricing = pd.read_csv('https://raw.githubusercontent.com/aaiken1/fin-data-analysis-python/main/data/train_2016_v2.csv')

zillow_data = pd.merge(housing, pricing, how='inner', on='parcelid')
zillow_data['transactiondate'] = pd.to_datetime(zillow_data['transactiondate'], format='%Y-%m-%d')


In [ ]:
zillow_data.describe()

I'll print a list of the columns, just to see what our variables are. There's a lot in this data set.

In [ ]:
zillow_data.columns

Let's run a really simple regression. Can we explain pricing errors using the size of the house? I'll take the natural log of `calculatedfinishedsquarefeet` and use that as my independent (**X**) variable. My dependent (**Y**) variable will be `logerror`. I'm taking the natural log of the square footage, in order to have what's called a "log-log" model.

In [ ]:
zillow_data['ln_calculatedfinishedsquarefeet'] = np.log(zillow_data['calculatedfinishedsquarefeet'])

results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet", data=zillow_data).fit()


In [ ]:
print(results.summary())

One y variable and one X variable. That's the full summary of the regression. This is a "log-log" model, so we can say that a 1% change in square footage leads to a 1.39% increase in pricing error. The coefficient is positive and statistically significant at conventional levels (e.g. 1%). 

We can pull out just a piece of this full result if we like. 

In [ ]:
results.summary().tables[1]

We can, of course, include multiple **X** variables in a regression. I'll add bathroom and bedroom counts to the regression model.

Pay attention to the **syntax** here. I am giving `smf.ols` the name of my data frame. I can then **write the formula** for my regression using the names of my columns (variabes or features).

In [ ]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + bathroomcnt + bedroomcnt", data=zillow_data).fit()
print(results.summary())

Hey, all of my significance went away! Welcome to the world of [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity). All of these variables are very correlated, so the coefficient estimates become difficult to interpret.

We're going to use **machine learning** below to help with this issue.

Watch what happens when I just run the model with the bedroom count. The $t$-statistic is quite large again.

In [ ]:
results = smf.ols("logerror ~ bedroomcnt", data=zillow_data).fit()
print(results.summary())

## Indicators and categorical variables

The variables used above are measured numerically. Some are **continuous**, like square footage, while others are **counts**, like the number of bedrooms. Sometimes, though, we want to include an **indicator** for something? For example, does this house have a pool or not?

There is a variable in the data called `poolcnt`. It seems to be either missing (NaN) or set equal to 1. I believe that a value of 1 means that the house has a pool and that `NaN` means that it does not. This is bit of a tricky assumption, because `NaN` could mean no pool or that we don't know either way. But, I'll make that assumption for illustrative purposes.

In [ ]:
zillow_data['poolcnt'].describe()

I am going to create a new variable, `pool_d`, that is set equal to 1 if `poolcnt >= 1` and 0 otherwise. This type of 1/0 categorical variable is sometimes called an **indicator**, or **dummy** variable. 

This is an example of making the indicator variable **by hand**. I'll use `pd.get_dummies` below in a second.

In [ ]:
zillow_data['pool_d'] = np.where(zillow_data.poolcnt.isnull(), 0, zillow_data.poolcnt >= 1)
zillow_data['pool_d'].describe()

I can then use this 1/0 variable in my regression.

In [ ]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + pool_d", data=zillow_data).fit()
print(results.summary())

Pools don't seem to influence pricing errors. 

We can also create more general **categorical** variables. For example, instead of treating bedrooms like a count, we can create new categories for each number of bedrooms. This type of model is helpful when dealing states or regions. For example, you could turn a zip code into a categorical variable. This would allow zip codes, or a location, to explain the pricing errors. 

When using `statsmodel` to run your regressions, you can turn something into a categorical variable by using `C()` in the regression formula. 

I'll try the count of bedrooms first.

In [ ]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + C(bedroomcnt)", data=zillow_data).fit()
print(results.summary())

And here are zip codes as a categorical variable. This is saying: Is the house in this zip code or no? If it is, the indicator for that zip code gets a 1, and a 0 otherwise. If we didn't do this, then the zip code would get treated like a numerical variable in the regression, like square footage, which makes no sense!

In [ ]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + C(regionidzip)", data=zillow_data).fit()
print(results.summary())

**Think about these categorical variables as switches**. We also call them **dummy or indicator variables** when they are constructed in a 1/0 manner. If the variable is 1, then that observation has that characteristic. The predicted value changes by the coefficient if the indicator is 1. The switch is "turned on"./ 


### Using get_dummies

`pandas` also has a method called `pd.get_dummies`. Here's description from Claude (my emphasis):

pd.get_dummies() is a function in the Pandas library that is used to transform categorical variables into a format that can be used in machine learning models.

The function takes a Pandas DataFrame or Series as input and creates a new DataFrame where each unique categorical value is represented as a new column. The new columns are binary, with a value of 1 if the observation had that categorical value, and 0 otherwise.

This is a common preprocessing step for many machine learning algorithms that require numeric input features. By converting categorical variables into this dummy or one-hot encoded format, the machine learning model can better understand and incorporate the information from those categorical variables.

For example, if you had a DataFrame with a 'color' column that contained the values 'red', 'green', and 'blue', pd.get_dummies() would create three new columns: 'color_red', 'color_green', and 'color_blue', each with 0s and 1s indicating the color of each observation.

**This transformation allows the machine learning model to treat these categorical variables as distinct features, rather than trying to interpret them as ordinal or numeric data**. Using pd.get_dummies() is a crucial step in preparing many datasets for modeling.

One thing to note - it creates a new dataframe that you need to add back to the original data.

Here's some code, also using *bedroomcnt*.

In [ ]:
# Create categorical variable based on LotShape
bedroomcnt_categorical = pd.get_dummies(zillow_data['bedroomcnt'], prefix='bedroomcnt')

# Concatenate the categorical variable with the original DataFrame
zillow_data = pd.concat([zillow_data, bedroomcnt_categorical], axis=1)

I created the new columns and added them to the original Zillow dataframe. Each is an indicator (1/0) variable for whether or not the house has that number of bedrooms. If it does, we flip the switch and the coefficient tells us if a house with that number of bedrooms has larger or smaller pricing errors.

In [ ]:
zillow_data.columns

In [ ]:
zillow_data

If you are creating a feature data frame that **only includes exactly what you need**, you should drop the original variable once you've created the indicators that you need. In this case, you would drop *bedroomcnt*. 

`pd.get_dummies` has an option `dtype=` for the output. You can choose to get **booleans** (True or False) or **integers** (1/0) for the output. You'll want integers for your regression models. 

## Hull, Chapters 3 and 10. Linear regression and prediction.

Chapter 3.1 starts off with the math behind linear regression with one feature. This is just what we did above, when we tried to explain pricing errors with one variable. This math and how regression works should be familiar from your introductory statistics course. 


### Machine learning and prediction

We're going to introduce **machine learning concepts** now. Notice the idea of a **training set**. This is the data that we use to create our model. We are then going to take the outputs (i.e. the coefficients) from this model and use it to predict values in the **out-of-sample** data.

In machine learning applications, we have an **X** dataframe and a **y** dataframe. The **X** dataframe has all of our features. They are typically standardized. This means doing something like taking every numerical feature and turning it into a **z-score**. 

The **y** is the **target** variable. The thing that you are trying to predict. You'll also standardize this, typically. 

And, crucially, the **X** data frame **only** contains the features we want. No extra columns. Nothing that we used earlier to create a new variable. If it is in **X**, then it is going into the model. This is different from using other packages or languages, where you might write out your regression model. In fact, I do that above when using `statsmodel`. 

**Why don't people write out their models in machine learning? Too many variables and the algo is picking your model for you!**

The also textbook introduces the idea of **three different types of data sets: training, validation, and testing**. Let's ask Claude about the differences:

In machine learning, data is typically split into three subsets: training data, validation data, and test data. Each subset serves a different purpose in the model development and evaluation process.

**Training Data**: The training data is the largest portion of the dataset, and it is used to train the machine learning model. During the training process, the model learns patterns and relationships from the training data. The model's parameters are adjusted to minimize the error or loss function on this data.

**Validation Data**: The validation data is a subset of the data held out from the training process. It is used to evaluate the performance of the model during the training phase. The validation data provides an unbiased estimate of the model's performance on unseen data. It helps tune the **hyperparameters** of the model, such as the learning rate, regularization strength, or the number of iterations. The validation data is used to prevent **overfitting**, which occurs when the model performs well on the training data but fails to generalize to new, unseen data. By monitoring the performance on the validation data, early stopping or regularization techniques can be applied to improve the model's generalization capability.

**Test Data**: The test data is a separate subset of the data that is completely isolated from the training and validation processes. It is used to evaluate the final performance of the trained model on unseen data, providing an unbiased estimate of the model's real-world performance. The test data should be representative of the data the model will encounter in production or real-world scenarios. The performance metrics calculated on the test data, such as accuracy, precision, recall, or F1-score, are considered the final, reliable indicators of the model's effectiveness.

It is crucial to keep the test data separate and untouched during the training and validation phases to avoid any data leakage or overfitting to the test data. The test data should only be used once, after the model has been fully trained and tuned, to obtain an unbiased estimate of its performance.

By splitting the data into these three subsets, machine learning practitioners can train models effectively, tune hyperparameters, and evaluate the final performance objectively, ensuring that the model will generalize well to new, unseen data.

As you'll see, sometimes we only use two separate data sets, training and test data. We'll validate the model within the training data. 

`sklearn` has many built-in features to help, like ways to automatically split your data, though the textbook tends to do things manually. 

Chapter 3.2 introduces **multiple features**. Again, this just means multiple explanatory X variables. Eq. 3.1 gives us the general form of a linear regression model:

\begin{align}
Y = a + b_1 X_1 + b_2 X_2 + \cdots + b_m X_m + \varepsilon
\end{align}

The [statsmodel](https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html) page for linear regression has helpful code examples, including how to generate dummy variables (see below).


### Important regression statistics

What is regression trying to do? Hull discusses these linear models from the machine learning perspective. So, rather than a discussion of the [Gauss-Markov theorem and BLUE](https://en.wikipedia.org/wiki/Gauss–Markov_theorem), we get something less theoretical. 

In short, the linear regression algorithm selects values for $a$ and each $b$ above that minimize the sum of the squared differences between each Y and its predicted value (i.e. the right-hand side of the equation). This is shown in Eq. 3.2. Why do we square the differences? The direction of the error doesn't matter - we want predicted values to be close to the actual value. 

The text notes that $a$ is called the model **bias** and the $b$ values are called the model **weights**. In econometrics, we call $a$ the **intercept** and the $b$ values **coefficients**. 

Machine learning techniques are all about **prediction**. How well does the model predict or fit the data? To measure this, we can use the **R-Squared** value for the model. 

\begin{align}
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
\end{align}

We can also put this into words:

\begin{align}
R^2 = 1 - \frac{\text{Variance of the errors } \epsilon}{\text{Variation of the observations on the target Y}}
\end{align}

As noted by the text:

> The R-squared statistic is between zero and one and measures the proportion of the variance in the target that is explained by the features.

This is a statistics for the entire model. We also care about how well each feature (variable) does in explaining the variation in our outcome/target/Y variable. We use *t*-statistics for that. 

Again, from the text:

> The *t*-statistic of the $a$ or $𝑏_𝑗$ parameters estimated by a linear regression is the value of the parameter divided by its standard error. The P-value is the probability of obtaining a *t*-statistic as large as the one observed if we were in the situation where the parameter had no explanatory power at all. A P-value of 5% or less is generally considered as indicating that the parameter is significant. If the P-value for the parameter $b_j$ in equation (3.1) is less than 5%, we are over 95% confident that the feature $X_j$ has some effect on Y. The critical t-statistic for 95% confidence when the data set is large is 1.96 (i.e., t-statistics greater than  1.96 are significant in the sense that they give rise to P-values less than 5%).

I would take issue with some of the language in that explanation. These values are about relationships and correlations, but I see *effect* in that description. Saying that these correlations are actually causal relationships requires assumptions that go beyond pure statistics. You need a model. In our case, this would mean an economic model that describes incentives and behavior. 


### Categorical features, again

We saw these above, when we used *bedroomcnt*. From the Hull text:

> A common practice is to use the same number of dummy variables as there are categories and associate each dummy variable with a particular category. The value of a dummy variable is one if the corresponding category applies and zero otherwise. This is referred to as **one-hot encoding**.

We used `pd.get_dummies` above to one-hot encode some indicators.

The text also discusses the **dummy variable trap**. If you create a dummy variable for each categorical value and then add a constant to the regression model, your model is **overfit** and will not estimate. In economics, we usually include a constant or intercept term in our regressions. This isn't always the case in machine learning. 

To avoid this trap, **drop one of the indicators for the set**. So, suppose that you've taken a variable called *Color* that gets your three indicators for *Color_Red*, *Color_Green*, or 
*Color_Blue*. **Every** observation has a 1 in one of those columns and zeroes in the others. Drop one of the indicators, like *Color_Red* - remove the column from your data frame. Your model will now run.

By doing this, the *Color_Red* observations are those where *Color_Green* and *Color_Blue* are both zero. *Color_Green* and *Color_Blue* are estimated relative to *Color_Red*. For example, if the coefficient for *Color_Green* is 0.5, the the y-variable is shifted by 0.5 when *Color_Green* is equal to one, relative to the default prediction for the *Color_Red*.

As noted above, though, in machine learning, we often don't spend a lot of time interpreting the coefficients like this. Instead, we want to include as much information as possible in order to make a prediction. Get your model to run and focus on the predicted y variables.


### Regularization and Ridge regression

These next three methods come from the machine learning literature and help to deal with having **too many explanatory variables**. When I say this, I mean that they come more from the computer science and statistics world than the econometrics world. This distinction is meaningful if you're trained in one set of methods vs. another, but is becoming less important over time as machine learning and econometrics borrow from each other. 

```{margin} What is econometrics?
Econometrics is usually interested in causal relationships among economic agents. So, does more schooling increase wages, or do the people who are good at school also earn more? These are difficult questions to answer and economists have developed a variety of tools to help. If you've taken econometrics, then you've seen topics like instrumental variables and difference-in-differences. In economics, we're more interested in explaining why something is happening than predicting an outcome. So, did a change in a zoning law affect housing prices, rather than predicting a pricing error. 
```

Why are too many explanatory (X) variables a problem? They make the math harder when different X variables are correlated with one another. In other words, we have different features in our data that are really explaining the same thing. The book uses the area of the basement and the area of the first floor of a house as an example. These values are different, but capture the same thing, generally speaking - house size. 

**Regularization** helps use reduce the number of variables used in our model. We are also going to see more **scaling** of features. It is important to scale (e.g. Z-score) in order for different features to be comparable with each other. 

A larger, positive z-score means that the observation is above the mean value for that feature (X) or target (y). Vice-versa for negative values. Values are centered around zero after scaling this way. As Hull notes, there are other ways to scale variables, but this Z-score method is very common.

Here's a [sklearn article](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html) on the importance of scaling your features before using them in a machine learning model. From the article:

> Feature scaling through standardization, also called Z-score normalization, is an important preprocessing step for many machine learning algorithms. It involves rescaling each feature such that it has a standard deviation of 1 and a mean of 0.

One way to regularize our model is to use a **Ridge regression**. This algorithm chooses $a$ and $b$ parameters to minimize this function:


\begin{align}
\frac{1}{n}\sum_{i=1}^{n}(Y_i - a - b_1X_{i1} - b_2X_{i2} - \cdots - b_mX_{im})^2 + \lambda\sum_{j=1}^{m}b_j^2
\end{align}

Note that this looks a lot like a normal regression, except that we now have the last term with $\lambda$ and $b^2$. From the text:

> This change has the effect of encouraging the model to keep the weights $b_j$ as small as possible.

This method is also called **L2 regularization**. Why do we do this? From the text:

> L2 regularization is one way of avoiding the situation such as that in equation (3.3) where there are two correlated features, one with a large positive weight and the other with a large negative weight. Its general effect is to **simplify a linear regression model**. Sometimes, the model obtained after regularization generalizes better than the model before regularization. 

How do we choose a value for $\lambda$? From the text:

> The parameter $\lambda$ is referred to as a **hyperparameter** because it is used as a tool to determine the best model but is not part of the model itself. The choice of a value for $\lambda$ is obviously important. A large value for λ would lead to all the $b_j$ being set equal to zero. (The resulting model would then be uninteresting as it would always predict a value equal to $a$ for Y.) In practice, it is desirable to try several different values for $\lambda$ and choose between the resulting models.

If $\lambda$ is set to zero, then we have linear regression. As you increase $\lambda$, the model will become **less complex**.

Take a look at **footnote 3**. `sklearn` uses the **sum of squared errors** rather than the **mean sum of squared errors**. In other words, there's no $\frac{1}{n}$ out in front of the Ridge regression formula. So, you have to multiple your lambda term by $n$, or the number of observations, when using it. See more below, where I work the example in the Hull text.



### LASSO regression

From the Hull text:

> In Ridge, we added a constant times the sum of the squared weights to the objective function. In LASSO, we add a constant times the sum of the **absolute weights**.

LASSO is also called **L1 regularization**. This changes the last term in the equation that we are trying to minimize:

\begin{align}
\frac{1}{n}\sum_{i=1}^{n}(Y_i - a - b_1X_{i1} - b_2X_{i2} - \cdots - b_mX_{im})^2 + \lambda\sum_{j=1}^{m}|b_j|
\end{align}

I asked the Claude GPT to explain this equation:

This equation represents the objective function for LASSO (Least Absolute Shrinkage and Selection Operator) regression. LASSO is a type of linear regression that uses L1 regularization to add a penalty term to the cost function. This penalty term encourages sparse solutions, driving some coefficient estimates to exactly zero, effectively performing variable selection and improving interpretability of the model. The first term is the regular least squares error term that measures the model's fit to the data. The second term is the L1 regularization penalty on the sum of the absolute values of the coefficients ($|b_j|$), multiplied by a tuning parameter lambda that controls the strength of the penalty.

By minimizing this objective function, LASSO regression aims to find the best fit linear model while also promoting sparsity in the coefficients, making it useful for scenarios with many potential predictor variables.

Using LASSO is going to mean having more $b = 0$ values. In other words, some features are "kicked out of the model" in order to simplify things. This is going to happen when **certain features explain the same variation in the Y variable**. 

Again, check out footnote 4 for some details on how lASSO works in `sklearn`. 



### Elastic net regression

Elastic Net regression is a **mixture of Ridge and LASSO**. The function to be minimized includes both a constant times the sum of the squared weights and a different constant times the sum of the absolute values of the weights.

\begin{align}
\frac{1}{n}\sum_{i=1}^{n}(Y_i - a - b_1X_{i1} - b_2X_{i2} - \cdots - b_mX_{im})^2 + \lambda_1\sum_{j=1}^{m}b_j^2 + \lambda_2\sum_{j=1}^{m}|b_j|
\end{align}

From the text:

> In LASSO some weights are set to zero, but others may be quite large in magnitude. In Ridge, weights are small in magnitude, but they are not reduced to zero. The idea underlying Elastic Net is that we may be able to get the **best of both worlds** by making some weights zero while reducing the magnitude of the others.

From Claude:

This equation represents the objective function for Elastic Net regularization, which is a linear regression model that combines the L1 (LASSO) and L2 (Ridge) regularization techniques. The first term is the standard mean squared error term, which measures the difference between the predicted values and the actual target values ($Y_i$) for the given data points ($X_{i1}$, $X_{i2}$, ..., $X_{im}$). 

The second term is the L2 regularization penalty, also known as the Ridge penalty, which adds the sum of the squared coefficients ($b_j^2$) multiplied by a tuning parameter $λ_1$. This penalty encourages small coefficient values and helps to prevent overfitting.

The third term is the L1 regularization penalty, also known as the LASSO penalty, which adds the sum of the absolute values of the coefficients ($|b_j|$) multiplied by a tuning parameter $λ_2$. This penalty promotes sparsity in the model by driving some coefficients to exactly zero, effectively performing variable selection.

By combining the L1 and L2 penalties, Elastic Net regularization aims to benefit from the strengths of both Ridge and LASSO regularization. It can handle correlated predictors better than LASSO and can select more predictive features than Ridge regression. The tuning parameters $λ_1$ and $λ_2$ control the relative importance of the L1 and L2 penalties, respectively. Elastic Net regularization is particularly useful when dealing with high-dimensional data with potentially correlated predictors, where both variable selection and coefficient shrinkage are desirable.



### Using our tools to predict housing prices

I'm going to replicate the Hull textbook results below. This data has housing sale prices and features about each house. The houses are in Iowa and represent sales over a four year period. The data has already been scaled, so that each feature has a mean of 0 and a standard deviation of 1. As noted above, you should use the means and standard deviations from the training data to scale the testing data.

The text describes the data and its structure. We are going to create training, validation, and testing data sets. Note that the data are already cleaned up for you. If you were doing this on your own, you'd need to do the steps described on pg. 65. 

You do this in our lab assignments.

Notice how this is a different question from the Zillow data. We're directly predicting housing prices here, not Zillow's pricing errors. Zillow was trying to identify the types of houses that were difficult to price using their Zestimate model.

In [ ]:
# Both features and target have already been scaled: mean = 0; SD = 1
data = pd.read_csv('https://raw.githubusercontent.com/aaiken1/fin-data-analysis-python/main/data/Houseprice_data_scaled.csv') 

We are going to split this data into **training** and **validation** sets. We'll estimate the model on the training data and then check it using the validation set. Note the use of `.iloc`. 

In [ ]:
# First 1800 data items are training set; the next 600 are the validation set
train = data.iloc[:1800] 
val = data.iloc[1800:2400]

When we split the data like this, we're assuming that the data are already randomly shuffled. In practice, this probably isn't a great assumption. `sklearn` has built-in tools to do this.

We then separate the X and Y variables into do different data sets. This is the way the `sklearn` package wants things.

In [ ]:
# Creating the "X" and "y" variables. We drop sale price from "X"
X_train, X_val = train.drop('Sale Price', axis=1), val.drop('Sale Price', axis=1)
y_train, y_val = train[['Sale Price']], val[['Sale Price']] 

In [ ]:
y_train.head()

In [ ]:
X_train.head()

Now we can create the model. We are creating a linear regression object from the `sklearn` library. We then fit the model using our X and y training data. The results live inside of this object.

**See how there's no equation?** All of the features that I want to include are in the `X_train` data frame. My target variable is in `y_train`. I'm using the features to predict my target.

You can not do this step unless those two data frames are exactly what you want, no extra columns, everything is numeric, etc.

In [ ]:
lr = LinearRegression()

lr.fit(X_train, y_train)

In [ ]:
type(lr)

We can pull values out of this model, but they aren't easy to read. 

In [ ]:
lr.intercept_

In [ ]:
lr.coef_

Let's put that same information into an easier to read Dataframe. We can output the coefficients, or $b$ values, from the model, along with the intercept, or $a$ value.

In [ ]:
# Create dataFrame with corresponding feature and its respective coefficients
coeffs = pd.DataFrame(
    [
        ['intercept'] + list(X_train.columns),
        list(lr.intercept_) + list(lr.coef_[0])
    ]
).transpose().set_index(0)
coeffs

These results match what's on pg. 66 of the Hull text. Note that there are 25 neighborhood dummy variables. Basement quality ($BsmtQual$) is also a categorical variable with ordered values. 

We can get the **R Squared value from this model**. This also matches what's in the book. 

In [ ]:
lr.score(X_train,y_train)

By the way, `statsmodel` gives much nicer output. You have to manually include a constant/intercept term, the $a$ value, in the Hull text.

In [ ]:
#add constant to predictor variables
x = sm.add_constant(X_train)

#fit linear regression model
model = sm.OLS(y_train, x).fit()

#view model summary
print(model.summary())

I hope that output likes this is familiar! $LotArea$, $OverallQual$, $OverallCond$, and $YearBuilt$ all of strong positive relationships with sale price. That makes sense. However, note that there are some strange negative relationships, like $BedroomAbvGr$. The book notes some of these negative coefficients. This is because "kitchen sink" models like this, where you use everything to predict something, end up with a lot of **multicollinearity**. Our variables are related to each other. This can lead to strange coefficient values.  

### Making predictions

Now, let's use these model parameters ($a$ and $b$ values) to **predict** prices in the **validation** data set. To do this, we are going go back to `sklearn` and use `predict`. I am taking my linear regression object **lr**. This object contains the model that I estimated using my **training** data. It has all of the coefficients and the intercept. 

I can, in essence, multiply the coefficients by their corresponding features and get predicted values. You can see this regression linear algebra in the Hull text.

In [ ]:
# Make predictions using the validation set
y_pred = lr.predict(X_val)

# The coefficients
print("Coefficients: \n", lr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mse(y_val, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2(y_val, y_pred))


What is in `y_pred`? These are the predicted scaled housing prices using the house data that we "held out" in the validation sample. 

In [ ]:
y_pred[:10]

Now, those don't look like housing prices to me! That's because Hull already scaled the data in the original CSV file. These are **predicted, scaled values**. If we had scaled this data using a transformer from the `sklearn` package, then we could easily unscale them here and get real housing prices. I discuss scaling data more below.

We can compare, though, how the predicted values differ from the actual values. We have computed predicted values from the model coefficients using the test data and the X features held out of sample. But, remember, we also have the actual y values. Let's **compare the two graphically**. I'll use some `matplotlib`.

In [ ]:
# Create a figure and axis object
fig, ax = plt.subplots(figsize=(8, 6))

# Scatter plot of actual vs predicted values
ax.scatter(y_val, y_pred, alpha=0.5)

# Add a diagonal line for reference
ax.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=2)

# Set labels and title
ax.set_xlabel('Actual Values')
ax.set_ylabel('Predicted Values')
ax.set_title('Actual vs Predicted Values')

# Show the plot
plt.show();

### Ridge regression

Let's try a **Ridge regression** now. Note that `sklearn` uses an $\alpha$ term, rather than $\lambda$, like in the text. The two are related. See the footnotes in Chapter 3. 

This code is running different Ridge regressions with different $\alpha$ terms, reporting the mean squared error for each. The error will go up as $\alpha$ increases, but we're looking for values where that increase isn't as large.

In [ ]:
# The alpha used by Python's ridge should be the lambda in Hull's book times the number of observations
alphas=[0.01*1800, 0.02*1800, 0.03*1800, 0.04*1800, 0.05*1800, 0.075*1800,0.1*1800,0.2*1800, 0.4*1800]
mses=[]
for alpha in alphas:
    ridge=Ridge(alpha=alpha)
    ridge.fit(X_train,y_train)
    pred=ridge.predict(X_val)
    mses.append(mse(y_val,pred))
    print(mse(y_val,pred))

Let's plot $\alpha$ values on the y-axis and MSE on the x-axis. 

In [ ]:
plt.plot(alphas, mses)

Note that this isn't exactly what's in the text. Figure 3.8 has $\lambda$ on the x-axis and Variance Unexplained on the y-axis. This plot gives the same information, though. 

### Lasso regression

Let's look at a **lasso regression**. We create the lasso object with a specific alpha hyperparameter. Then, we feed that object the necessary data. We'll use our training data.

In [ ]:
# Here we produce results for alpha=0.05 which corresponds to lambda=0.1 in Hull's book
lasso = Lasso(alpha=0.05)
lasso.fit(X_train, y_train)

We can output the model using this bit of code. Remember, model specifications are contained in the object created when we fit, or estimate, the model.

In [ ]:
# DataFrame with corresponding feature and its respective coefficients
coeffs = pd.DataFrame(
    [
        ['intercept'] + list(X_train.columns),
        list(lasso.intercept_) + list(lasso.coef_)
    ]
).transpose().set_index(0)
coeffs

Do you see all of the zero values? The lasso model penalizes features that are either unimportant or similar to other variables in the model. Notice how all of the negative weights are gone.

Let's do the same thing we did above and vary our hyperparameter.

In [ ]:
# We now consider different lambda values. The alphas are half the lambdas
alphas=[0.01/2, 0.02/2, 0.03/2, 0.04/2, 0.05/2, 0.075/2, 0.1/2]
mses=[]
for alpha in alphas:
    lasso=Lasso(alpha=alpha)
    lasso.fit(X_train,y_train)
    pred=lasso.predict(X_val)
    mses.append(mse(y_val,pred))
    print(mse(y_val, pred))

In [ ]:
plt.plot(alphas, mses)

### Elastic net

The text comments that the elastic net model doesn't really give much of an improvement over lasso. Let's estimate it here, though.

In [ ]:
elastic = ElasticNet(alpha=0.05, l1_ratio=0.8, fit_intercept=True)
elastic.fit(X_train, y_train)

We again create our model object and then fit it with data.

What do the `alpha` and `l1_ratio` parameters do? From Claude:

In the scikit-learn implementation of Elastic Net regression (ElasticNet class), the alpha and l1_ratio parameters control the strength and balance between the L1 (Lasso) and L2 (Ridge) regularization penalties, respectively.

- alpha (float): This parameter controls the overall strength of the regularization. A higher value of alpha increases the regularization effect, shrinking the coefficients more towards zero. When alpha=0, there is no regularization, and the model becomes a ordinary least squares linear regression.

- l1_ratio (float between 0 and 1): This parameter determines the balance between the L1 and L2 penalties in the Elastic Net regularization. Specifically: l1_ratio=1 means the penalty is an L1 penalty (Lasso). l1_ratio=0 means the penalty is an L2 penalty (Ridge). 0 < l1_ratio < 1 means the penalty is a combination of L1 and L2 penalties (Elastic Net).

So, alpha controls the overall regularization strength, while l1_ratio determines how much of that regularization comes from the L1 penalty vs the L2 penalty. In practice, you would tune both alpha and l1_ratio using cross-validation to find the best combination for your specific data. Common values for l1_ratio are 0.5 (balanced Elastic Net), 0.2, or 0.8 depending on whether you want more L2 or L1 regularization respectively.

Let's look.

In [ ]:
# DataFrame with corresponding feature and its respective coefficients
coeffs = pd.DataFrame(
    [
        ['intercept'] + list(X_train.columns),
        list(elastic.intercept_) + list(elastic.coef_)
    ]
).transpose().set_index(0)
coeffs

Watch what happens when I set `l1_ratio` = 1. I get the lasso model!

In [ ]:
elastic = ElasticNet(alpha=0.05, l1_ratio=1, fit_intercept=True)
elastic.fit(X_train, y_train)

In [ ]:
coeffs = pd.DataFrame(
    [
        ['intercept'] + list(X_train.columns),
        list(elastic.intercept_) + list(elastic.coef_)
    ]
).transpose().set_index(0)
coeffs

## Prediction workflow with sklearn

The Hull textbook often does things "by hand". For example, he splits the data into training, validation, and test samples by using `.iloc` and manually slicing the data frame. This is problem if the data aren't sorted randomly! He standardizes the variables by using he z-score formula. He searches for optimal hyperparameters by making a graph.

It turns out the `sklearn` can do a lot of this for us. In fact, **if you use tools like Copilot or ChatGPT** you're likely to get code that uses the `sklearn` workflow. So, we should understand how to use these tools. The fact that ChatGPT wants to use them means that they are extremely common.

This chapter starts off with a discussion of the general workflow, so I'm assuming that you've already done your basic data cleaning.


### Splitting your data

Let's use `sklearn` and our Zillow data from above to **split the data into training and testing samples**. From Claude:

train_test_split is a function in the sklearn.model_selection module of the scikit-learn library in Python. It is used to split a dataset into two subsets: a training set and a test set. The training set is used to train a machine learning model, while the test set is used to evaluate its performance on unseen data.

The train_test_split function takes the following arguments:

- X: The feature matrix or data frame containing the input variables.
  
- y: The target vector or data frame containing the output variable(s).
  
- test_size (optional): The proportion of the dataset to include in the test split. It should be a float between 0.0 and 1.0. The default value is 0.25, which means 25% of the data will be used for testing.
  
- random_state (optional): An integer value used to seed the random number generator, which ensures reproducibility of the split.

The function returns four objects:

- X_train: The training data for the input features.
- X_test: The testing data for the input features.
- y_train: The training data for the target variable(s).
- y_test: The testing data for the target variable(s).

Notice that it doesn't do the train, validation, and test split from the Hull book. We can do training and validation within the training data set. I'll take my data frame *zillow_data* and split into target and features. Remember, *logerror* was the variable that we were trying to predict.

I added *z* to all of the names, just so I don't overwrite my data frames from above.

In [ ]:
zillow_sample = zillow_data[['logerror', 'calculatedfinishedsquarefeet', 'lotsizesquarefeet', 'yearbuilt']]

yz = zillow_sample['logerror']
Xz = zillow_sample.drop('logerror', axis=1)

Xz_train, Xz_test, yz_train, yz_test = train_test_split(Xz, yz, test_size=0.2, random_state=42)

That will create four new data frames for you. I would make sure that you've cleaned and standardized everything before you split your data.

AI always chooses 42 to seed its random number generated. The answer to everything.

### Standardizing your data

Let's again use `sklearn` and our Zillow data from above to **standardize our data**. From Claude:

In scikit-learn (sklearn), you can standardize your data using the StandardScaler class from the sklearn.preprocessing module. Standardization is a common preprocessing step in machine learning that scales the features to have a mean of 0 and a standard deviation of 1. This can be beneficial for algorithms that are sensitive to feature scales.

- scaler = StandardScaler(): Create an instance of the StandardScaler class.
  
- X_scaled = scaler.fit_transform(X): Fit the scaler to the data and transform the data in one step. The fit_transform method first computes the mean and standard deviation of each feature based on the data in X, and then scales the features by subtracting the mean and dividing by the standard deviation.

The fit_transform method returns the scaled feature matrix X_scaled, where each feature now has a mean of 0 and a standard deviation of 1.

I am going to keep a few variables from the Zillow data, including our target *logerror*.

`scaler.fit_transform` creates an **array**. I convert this array back to a **data frame** and give the data frame the same column names as before. 


In [ ]:
scaler = StandardScaler()
Xz_train_scaled = scaler.fit_transform(Xz_train)

Xz_train_scaled = pd.DataFrame(Xz_train_scaled, columns=Xz_train.columns)

Xz_train_scaled

Notice the steps above. I create the **scaler** object. It's defined using `StandardScaler()` which does the z-score method. I then use `.fit_transform()` with it. This method combines two tasks. First, it **fits the scaling method** by taking the mean and standard deviation for all of the features. Then, it **transforms my data** using the scaler. I save this in a new array, which I then turn back into a data frame.

We need to transform our y, or target variable as well. Again, we'll pay attention to data types. *yz_train* is a series, not a data frame. `scaler()` wants a data frame.

In [ ]:
type(yz_train)

In [ ]:
yz_train= pd.DataFrame(yz_train)

In [ ]:
yz_train

In [ ]:
yz_train_scaled = scaler.fit_transform(yz_train)

yz_train_scaled = pd.DataFrame(yz_train_scaled, columns=yz_train.columns)

yz_train_scaled

Now, what about the **testing data**? From Claude:

When scaling your test data using scikit-learn's scalers (e.g., StandardScaler, MinMaxScaler, etc.), **you should use the mean and standard deviation (or the minimum and maximum values for MinMaxScaler) computed from the training data**. This is to ensure that the test data is scaled using the same parameters as the training data, which is crucial for preventing data leakage and maintaining consistency between the training and testing data distributions.

Watch my steps here, as they are different from above. I again create my **scaler object**, calling it *scaler*. You could create multiple scalers with different names if you wanted to.

I then **fit this scaler using my unscaled training data**. This is what I did above, except there I used `.fit_transform`, which fits the scaler and then transforms the data, all in one step. Now, I'm **fitting my scaler with one data set and then transforming a different data set**.

I then take this fitted scaler that has the means and standard deviations from my **training data**. I use those to **transform my testing data**. 

In [ ]:
scaler = StandardScaler()
scaler.fit(Xz_train)
Xz_test_scaled = scaler.transform(Xz_test)


I follow the same logic for my y target test data.

In [ ]:
yz_test= pd.DataFrame(yz_test)
yz_test

In [ ]:
yz_train

In [ ]:
scaler = StandardScaler()
scaler.fit(yz_train)
yz_test_scaled = scaler.transform(yz_test)

### Cross-validation and hyperparamters

Let's discuss an idea called [cross validation](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html). We've already see this a bit when we split our data into **training** and **validation** samples. From Claude:

> Cross-validation is a technique used in machine learning to evaluate the performance of a model and tune its hyperparameters. It helps to estimate how well the model will generalize to unseen data, and it is particularly useful when the available data is limited. The main idea behind cross-validation is to split the dataset into two parts: a training set and a validation set. The model is trained on the training set and evaluated on the validation set. This process is repeated multiple times, with different portions of the data being used for training and validation in each iteration.

> There are several types of cross-validation techniques, but the most common is **k-fold cross-validation**. Here's how it works:

1. The dataset is randomly partitioned into k equal-sized subsets or "folds".

2. One of the k folds is held out as the validation set, and the remaining k-1 folds are used for training the model. The model is trained on the k-1 folds and evaluated on the held-out validation fold.

3. This process is repeated k times, with each of the k folds being used as the validation set exactly once.

4. The performance metric (e.g., accuracy, F1-score, mean squared error) is calculated for each iteration, and the final performance is the average of the k iterations.

5. Cross-validation has several advantages:

- It provides a more reliable and unbiased estimate of the model's performance compared to a single train-test split.

- It helps to prevent overfitting by evaluating the model on data that it has not seen during training.

- It allows for effective hyperparameter tuning by selecting the hyperparameters that yield the best average performance across the cross-validation folds.

- It makes efficient use of the available data by using every data point for both training and validation at some point.

- Cross-validation is widely used in machine learning competitions, research, and real-world applications. It is a crucial technique for evaluating and tuning models, especially when dealing with limited data or when it is important to estimate the model's generalization performance accurately.

We are going to use some **built-in cross-validation tools**. This means that we don't have to manually split our data set. 

I'm going to go back the the Hull data and **combine the training and validation data sets**. I'm going to use all of the observations and have `sklearn` perform the cross-fold validation for me. 

Let me create my new data frames. This is the Hull data, so it's already cleaned up and standardized.

In [ ]:
combined_train = pd.concat([train, val], ignore_index=True)

X_combined_train = combined_train.drop('Sale Price', axis=1)
y_combined_train= combined_train[['Sale Price']]

I will now search for the optimal hyperparamter (i.e. alpha) value for a LASSO model using **cross-validation**. I am going to take each alpha (e.g. 0.001) and try it five times in my training data. Each time, I will "hold out" 20% of my training data, estimate a model on 80% of the data using that alpha value. I'll save my R-Square value from that model as a measure of fit. I'll then do that four more times for that alpha parameter, holding out a different 20% of my training data each time. That will give me five different R-Square values for each alpha. I'll then take an average of those R-Square values as my estimate for how well that alpha value performs. Remember, a larger R-Square means that the model "fits" the data better - the model is better able to use your features to explain your targets.

This process is repeated for each alpha value.

We are going to use `GridSearchCV` to do this. From Claude:

GridSearchCV is a class in scikit-learn that helps you perform an exhaustive search over specified parameter values for an estimator (a machine learning model or algorithm). It is used to find the optimal combination of hyperparameters that maximizes the performance of a machine learning model on a given dataset.



And a visual explanation of the process.

```{figure} ../images/12-cross-validation.png
---
name: 12-cross-validation.png
align: center
---
Holding out different parts of our training data in order to search for optimal hyperparameters. Source:Justintodata.com
```

In [ ]:
# Tune the LASSO hyperparameter (alpha) using the validation set
param_grid = {'alpha': [0.001, 0.01, 0.02, .05, 0.1]}
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2')
grid_search.fit(X_combined_train, y_combined_train)

# Extract the R-squared scores for each fold and hyperparameter combination
r2_scores = grid_search.cv_results_['mean_test_score']

# Print the R-squared scores
print(r2_scores)

As we already knew, lower alphas give us better results in our training data. We can use this method to **select a single set of hyperparameters** to then use on our **testing data**.

## Pipelines

`sklearn` lets us combine many of these steps together. I'm going to [follow along with this example](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html) and try something a little fancier. We will use some different model fit statistics to perform cross-validation within our training sample and have the pipeline **choose our hyperparameters for us**. 

`pipelines` can do much, much more that I show here. Take a look: <https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html>

Since I want to use some data that hasn't been standardized yet, I'll go back to the Zillow example. I'll keep some more numerical variables that will be easy to standardize.

I will not include any indicators or other data transformations. If I was doing this for real, I would spend a lot more time doing some feature engineering.

A few things to note:

- I'm using `make_pipeline`. This is a very common and useful function that let's you take your features, scale them, define criteria for model selection, etc. You basically transform your data in a series of steps and then ask for the model. 
  
- I'm asking for the [aic criterion](https://en.wikipedia.org/wiki/Akaike_information_criterion). This is a measure of best fit. I will also ask for the [bic criterion](https://en.wikipedia.org/wiki/Bayesian_information_criterion), another measure of best fit. The smaller the value, the better the model fits the data.
  
- `LassoLarsIC` tells the code to run the lasso model, which we then fit on our training data. This function is going to pick a bunch of alpha values for us, fit the model, and report the requested criterion back to us.


Here's a description of `make_pipeline` from Claude GPT:

- `make_pipeline` is a utility function provided by scikit-learn that creates a Pipeline object, which is a sequence of transformers and an estimator. It is a convenient way to chain multiple estimators into a single unit. The make_pipeline function takes any number of estimators as arguments and returns a Pipeline object. The estimators are applied in the order they are passed to the function. The last estimator is always a classifier or regressor, while the other estimators are transformers (preprocessors).

- Some key features and benefits of using make_pipeline are:

- **Convenience**: It provides a simple and readable way to define a sequence of estimators, making the code more concise and easier to maintain.

- **Automated Data Flow**: The Pipeline object automatically applies the transformers to the input data and passes the transformed data to the final estimator. This avoids the need to manually handle the data flow between estimators.

- **Consistency**: All estimators in the pipeline, except the last one, must be transformers (i.e., have a transform method). This consistency makes it easier to understand and reason about the data flow.

- **Persistence**: The entire pipeline can be persisted to disk using the joblib module, making it easy to reuse the same preprocessing and modeling steps in different environments.

- **Parallelization**: Some transformers in the pipeline can be parallelized using the n_jobs parameter, potentially leading to faster processing times.

- **Cross-Validation**: The entire pipeline can be used in cross-validation loops, ensuring that the same preprocessing steps are applied to both the training and validation data.

- Overall, make_pipeline simplifies the process of chaining multiple estimators together and helps to create more readable, maintainable, and efficient code for machine learning workflows.

Here's my `make_pipeline`. I am fitting a LASSO model, where I have it use the AIC criteria to pick the "best" hyperparameter. It then fit the model using our new combined training data.

There's a little bit of data manipulation that `make_pipline` wants me to do. From Claude:

y_train.values converts the pandas DataFrame y_train to a NumPy array.
.ravel() is then applied to the NumPy array, which flattens the array into a 1-dimensional array (required by the LassoLarsIC estimator).

I need to do a little data cleaning on this Zillow data to get it to work in our pipeline.


In [ ]:
zillow_sample = zillow_data[['logerror','bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'fireplacecnt', 
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft','lotsizesquarefeet', 
       'poolcnt', 'poolsizesum', 'yearbuilt', 'numberofstories']]


In [ ]:
zillow_sample

For some of these counts, missings are actually zeroes. I can glance at the data and see, however, that this isn't true for all observations. For example we have a poolcnt = 1, but poolsizesum = 0. I'll go ahead and just fill in the missings for the purpose of this exercise. But, not great!

In [ ]:
# Set missing observations to zero
zillow_sample = zillow_sample.fillna(0)

zillow_sample

In [ ]:
yz = zillow_sample.loc[:, 'logerror'] 
Xz = zillow_sample.drop('logerror', axis=1)

Xz_train, Xz_test, yz_train, yz_test = train_test_split(Xz, yz, test_size=0.2, random_state=42)

In [ ]:
Xz_train

In [ ]:
Xz_test

In [ ]:
yz_train

In [ ]:
yz_test

Time to make the `pipeline`. This is going to use the standard scaler on my training data and run a Lasso model, here the hyperparamter is selected using the AIC criteria.

Again, note how the X features set is are data frames, while the y target set are series.

In [ ]:
lasso_lars_ic = make_pipeline(StandardScaler(), LassoLarsIC(criterion="aic")).fit(Xz_train, yz_train.values.ravel())

In [ ]:
results = pd.DataFrame(
    {
        "alphas": lasso_lars_ic[-1].alphas_,
        "AIC criterion": lasso_lars_ic[-1].criterion_,
    }
).set_index("alphas")
alpha_aic = lasso_lars_ic[-1].alpha_

Same thing as above, but now using the BIC selection criteria.

In [ ]:
lasso_lars_ic.set_params(lassolarsic__criterion="bic").fit(Xz_train, yz_train.values.ravel())
results["BIC criterion"] = lasso_lars_ic[-1].criterion_
alpha_bic = lasso_lars_ic[-1].alpha_

You can see below that small values of alpha give us the best model fit, no matter the criteria used. This confirms what we saw graphically above. 

In [ ]:
def highlight_min(x):
    x_min = x.min()
    return ["font-weight: bold" if v == x_min else "" for v in x]

results.style.apply(highlight_min)

This table highlights the optimal alpha hyperparameter based on two different selection criteria.

Let's make a plot. We'll use `matplotlib` and the *ax* method.

In [ ]:
ax = results.plot()
ax.vlines(
    alpha_aic,
    results["AIC criterion"].min(),
    results["AIC criterion"].max(),
    label="alpha: AIC estimate",
    linestyles="--",
    color="tab:blue",
)
ax.vlines(
    alpha_bic,
    results["BIC criterion"].min(),
    results["BIC criterion"].max(),
    label="alpha: BIC estimate",
    linestyle="--",
    color="tab:orange",
)
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel("criterion")
ax.set_xscale("log")
ax.legend()
_ = ax.set_title(
    f"Information-criterion for model selection"
)

This doesn't look as continuous as it will in practice. The lines are jumping around quite a bit. The reason: We actually don't have that many features in this Zillow data set that I'm using. Our machine learning tools will work, but are probably unnecessary. Just run a regression!

Let's create a new pipeline using **cross-validation**, rather than using the AIC or BIC criteria. What's the difference? From Claude:

**LassoCV** uses cross-validation to find the optimal alpha value that minimizes a chosen scoring metric, making it more robust but computationally expensive.
**LassoLarsIC(criterion="aic")** uses the Akaike Information Criterion (AIC) to find the optimal alpha value, providing a faster but potentially less accurate estimate than cross-validation.

In [ ]:
modelcv = make_pipeline(LassoCV(cv=20)).fit(Xz_train, yz_train)

Again, *modelcv* is a pipeline. A pipeline is an object. This object contains our lasso model fit. We'll pull this out below and define it as *lasso*. 

In [ ]:
modelcv

In [ ]:
lasso = modelcv[-1]
lasso

OK, what did we do here? Let's have Claude tell us:

In scikit-learn, LassoCV(cv=20) refers to the Lasso regression model with cross-validation for tuning the regularization parameter (alpha). Here's what the components mean:

- LassoCV: This is the class name for the Lasso regression model with built-in cross-validation for automatically tuning the regularization strength parameter (alpha).

- cv=20: This is a parameter that specifies the number of folds for cross-validation. In this case, it means that the cross-validation process will be repeated 20 times, each time using a different 1/20th of the data as the validation set and the remaining 19/20ths as the training set.

- The LassoCV class performs the following steps:

1. It splits the data into cv (20 in this case) folds or subsets.

2. For each fold:

2a. It holds out that fold as the validation set.

2b. It trains the Lasso regression model on the remaining cv-1 folds, using a range of different alpha values.

2c. It evaluates the performance of the model on the held-out validation fold for each alpha value.

- After all folds have been processed, it selects the alpha value that gives the best average performance across all folds.

- Finally, it retrains the Lasso regression model on the entire dataset using the optimal alpha value found through cross-validation.

By using cross-validation, LassoCV can automatically find the optimal value of the regularization parameter (alpha) that balances the trade-off between model complexity and goodness of fit. This helps to prevent overfitting and improve the model's generalization performance. The choice of cv = 20 is a common practice, as it provides a good balance between computational cost and reliable estimation of the model's performance. However, the optimal value of cv can depend on the size of the dataset and the complexity of the problem.

I can print out the parameters contained in my LASSO model. This will tell you the details of how things were estimated.

In [ ]:
lasso.get_params()


I can also pull out the hyperparamter that the eventually choose using cross-validation.

In [ ]:
lasso.alpha_

That's a large hyperparameter. Lots of features were dropped. Let's look.

I'll make a table that has the coefficients from the final model estimated using my training data. Square feet of house and lot are all that are selected. The LASSO model drops every other feature.

In [ ]:
feature_names = list(Xz_train.columns) 
coefficients = lasso.coef_

# Create a DataFrame with feature names and coefficients
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort the DataFrame by the absolute value of coefficients in descending order
coef_df = coef_df.reindex(coef_df['Coefficient'].abs().sort_values(ascending=False).index)

coef_df

And, we can print the R2 for the final model from the training data. 

In [ ]:
score = modelcv.score(Xz_train, yz_train)
print(f"Test score: {score}")

Well, that's terrible! But, again, we're trying to predict **pricing errors** with only a subset of the features. Predicting actual housing prices is much easier.

I'm going to print my mse values in order to find some reasonable min and max values for the graph below. Remember, our target is the log of the pricing error. This isn't a dollar amount for the house.

See how I'm just pulling things out of my `lasso` object? It contains all of this information about our estimated model.

In [ ]:
lasso.mse_path_

In [ ]:
ymin, ymax = 0, 0.2

plt.semilogx(lasso.alphas_, lasso.mse_path_, linestyle=":")
plt.plot(
    lasso.alphas_,
    lasso.mse_path_.mean(axis=-1),
    color="black",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(lasso.alpha_, linestyle="--", color="black", label="alpha: CV estimate")

plt.ylim(ymin, ymax)
plt.xlabel(r"$\alpha$")
plt.ylabel("Mean square error")
plt.legend()
_ = plt.title(
    f"Mean square error on each fold: coordinate descent"
)

Finally, I'm going to use my fitted model from my training data on my testing data. I'll print the R2 value as a measure of how well we predict 

In [ ]:
# Evaluate the model on the test data
score = modelcv.score(Xz_test, yz_test)
print(f"Test score: {score}")


Still bad! I guess if you squint, it's a little better than what we did in the training data.

### Hyperparamters and model overfit

Again, what's the point of all of this? We're trying to find our model **hyperparameters** that we can then take to data outside of our training set. We are trying to avoid **overfitting**. This is a problem in machine learning and, especially, in financial applications. It is easy to look at past financial and fit a model that will predict returns **in sample**, in the data we are using. But, will this work on new data? If you overfit your model, you risk making precisely incorrect predictions.

Here's a [nice explanation](https://scikit-learn.org/stable/modules/cross_validation.html) on what is going on. This is all part of a much larger discussion on [model selection](https://scikit-learn.org/stable/model_selection.html)

### Interpreting linear regression models

Chapter 10.1 discusses how to interpret our results. Traditionally, machine learning hasn't been as interested in interpreting the model - we just want predicted values! However, as machine learning, statistics, econometrics, and specific domain knowledge (i.e. finance) mix, we are becoming more interested in exactly how to interpret our results.

As the test notes, you can't tell a loan applicant that they got rejected because "the algo said so"! You need to know why. This also gets into biases in our data. Is your model doing something illegal and/or unethical?

From the Hull text:

> The weight, $b_j$, can be interpreted as the sensitivity of the prediction to the value of the feature $j$. If the value of feature $j$ increases by an amount $u$ with all other features remaining the same, the value of the target increases by $b_{j,u}$. In the case of categorical features that are 1 or 0, the  weight gives the impact on a prediction of the target of changing the  category of the feature when all other features are kept the same. 

> The bias, $a$, is a little more difficult to interpret. It is the value of the target if all the feature values are zero. However, feature values of zero make no sense in many situations.

The text suggests setting all of the X features at the average value and then adding the intercept, $a$. This $a^*$ value is the predicted y if all features are at their mean. 

Note that to do this, you should use the model results with the unscaled data. From the text:

> To improve interpretability, the Z-score scaling used for Table 3.7 has been reversed in Table 10.1. (It will be recalled that scaling was necessary for Lasso.) This means that the feature weights in Table 3.7 are divided by the standard deviation of the feature and multiplied by the standard deviation of the house price to get the feature weights in Table 10.1.

As mentioned above, going back and forth between scaled and unscaled features is easier if you just let `sklearn` do it for you.

This is also a [nice summary](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html#sphx-glr-auto-examples-inspection-plot-linear-model-coefficient-interpretation-py) of how to interpret regression results.




### Some worked examples

We are just scratching the surface on machine learning in this course. Here are some complete examples that all use housing price data. Some things to pay attention to:

- How are the data organized? How is it split into training and test data?
- How is the data quality checked? How are missing values handled? How about outliers?
- What feature engineering steps are used? Are logs taken? Are there categorical variables in the regressions? Are the variables scaled?
- What visualizations are used to look at the data before modeling? Why are they used?
- What models are used? How are they compared?

[Example One.](https://www.kaggle.com/code/erick5/predicting-house-prices-with-machine-learning/notebook)

[Example Two.](https://www.dataquest.io/blog/kaggle-getting-started/)
